## Deliverable 2. Create a Customer Travel Destinations Map

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,jamestown,US,42.0970,-79.2353,35.01,93,90,9.22,light snow
1,1,albany,US,42.6001,-73.9662,37.99,67,97,14.56,overcast clouds
2,2,vokhtoga,RU,58.8147,41.0461,-9.45,88,41,2.84,light snow
3,3,santa fe,AR,-31.6333,-60.7000,81.00,29,0,8.99,clear sky
4,4,tongren,CN,27.7172,109.1853,38.91,75,36,2.06,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
matched_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
matched_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,santa fe,AR,-31.6333,-60.7000,81.00,29,0,8.99,clear sky
10,10,hithadhoo,MV,-0.6000,73.0833,81.57,71,86,4.92,overcast clouds
15,15,rikitea,PF,-23.1203,-134.9692,78.44,77,14,14.94,few clouds
19,19,arraial do cabo,BR,-22.9661,-42.0278,89.60,51,0,19.57,clear sky
20,20,samarai,PG,-10.6167,150.6667,82.99,80,45,13.60,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
matched_cities_df.notnull().sum()

City_ID                206
City                   206
Country                206
Lat                    206
Lng                    206
Max Temp               206
Humidity               206
Cloudiness             206
Wind Speed             206
Current Description    206
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#dropna
clean_df = matched_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,santa fe,AR,-31.6333,-60.7000,81.00,29,0,8.99,clear sky
10,10,hithadhoo,MV,-0.6000,73.0833,81.57,71,86,4.92,overcast clouds
15,15,rikitea,PF,-23.1203,-134.9692,78.44,77,14,14.94,few clouds
19,19,arraial do cabo,BR,-22.9661,-42.0278,89.60,51,0,19.57,clear sky
20,20,samarai,PG,-10.6167,150.6667,82.99,80,45,13.60,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
660,660,biak,ID,-0.9131,122.8766,76.03,86,100,6.11,light rain
661,661,quatre cocos,MU,-20.2078,57.7625,81.00,83,40,11.50,scattered clouds
662,662,cururupu,BR,-1.8283,-44.8683,88.29,57,38,7.34,scattered clouds
664,664,portobelo,PA,9.5500,-79.6500,90.00,75,38,8.99,scattered clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = None
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,santa fe,AR,81.00,clear sky,-31.6333,-60.7000,None
10,hithadhoo,MV,81.57,overcast clouds,-0.6000,73.0833,None
15,rikitea,PF,78.44,few clouds,-23.1203,-134.9692,None
19,arraial do cabo,BR,89.60,clear sky,-22.9661,-42.0278,None
20,samarai,PG,82.99,scattered clouds,-10.6167,150.6667,None
29,dingle,PH,75.34,light rain,10.9995,122.6711,None
34,cidreira,BR,80.38,overcast clouds,-30.1811,-50.2056,None
35,tabou,CI,80.60,light rain,4.4230,-7.3528,None
36,butaritari,KI,80.28,scattered clouds,3.0707,172.7902,None
37,atuona,PF,78.13,clear sky,-9.8000,-139.0333,None


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
        #Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
        
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")   

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,santa fe,AR,81.00,clear sky,-31.6333,-60.7000,Conquistador Hotel
10,hithadhoo,MV,81.57,overcast clouds,-0.6000,73.0833,Scoop Guest House
15,rikitea,PF,78.44,few clouds,-23.1203,-134.9692,Pension Maro'i
19,arraial do cabo,BR,89.60,clear sky,-22.9661,-42.0278,Pousada Porto Praia
20,samarai,PG,82.99,scattered clouds,-10.6167,150.6667,Nuli Sapi


In [13]:
#check for empty hotel values
hotel_df.count()

City                   206
Country                206
Max Temp               206
Current Description    206
Lat                    206
Lng                    206
Hotel Name             195
dtype: int64

In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(subset=["Hotel Name"])
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,santa fe,AR,81.00,clear sky,-31.6333,-60.7000,Conquistador Hotel
10,hithadhoo,MV,81.57,overcast clouds,-0.6000,73.0833,Scoop Guest House
15,rikitea,PF,78.44,few clouds,-23.1203,-134.9692,Pension Maro'i
19,arraial do cabo,BR,89.60,clear sky,-22.9661,-42.0278,Pousada Porto Praia
20,samarai,PG,82.99,scattered clouds,-10.6167,150.6667,Nuli Sapi
...,...,...,...,...,...,...,...
660,biak,ID,76.03,light rain,-0.9131,122.8766,Penginapan Kim
661,quatre cocos,MU,81.00,scattered clouds,-20.2078,57.7625,Emeraude Beach Attitude Hotel
662,cururupu,BR,88.29,scattered clouds,-1.8283,-44.8683,Pousada Través'cia
664,portobelo,PA,90.00,scattered clouds,9.5500,-79.6500,Scuba Portobelo


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df[["Max Temp"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))